In [1]:
!pip install mediapipe opencv-python numpy


In [3]:
pip install opencv-python

  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached numpy-2.2.4-cp311-cp311-win_amd64.whl.metadata (60 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl (39.5 MB)
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
    --------------------------------------- 0.3/12.9 MB ? eta -:--:--
   -- ------------------------------------- 0.8/12.9 MB 1.4 MB/s eta 0:00:09
   --- ------------------------------------ 1.0/12.9 MB 1.6 MB/s eta 0:00:08
   ---- ----------------------------------- 1.6/12.9 MB 1.7 MB/s eta 0:00:07
   ----- ---------------------------------- 1.8/12.9 MB 1.7 MB/s eta 0:00:07
   ------ --------------------------------- 2.1/12.9 MB 1.7 MB/s eta 0:00:07
   ------ --------------------------------- 2.1/12.9 MB 1.7 MB/s eta 0:00:07
   ------- -------------------------------- 2.4/12.9 MB 1.4 MB/s eta 0:00:08
   -------- ------------------

In [4]:
pip install mediapipe

  Using cached mediapipe-0.10.21-cp311-cp311-win_amd64.whl.metadata (10 kB)
  Using cached absl_py-2.2.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached jax-0.5.3-py3-none-any.whl.metadata (22 kB)
  Using cached jaxlib-0.5.3-cp311-cp311-win_amd64.whl.metadata (1.2 kB)
  Using cached matplotlib-3.10.1-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
  Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached protobuf-4.25.6-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached sounddevice-0.5.1-py3-none-win_amd64.whl.metadata (1.4 kB)
  Using cached sentencepiece-0.2.0-cp311-cp311-win_amd64.whl.metadata (8.3 kB)
  Using cached cffi-1.17.1-cp311-cp311-win_amd64.whl.metadata (1.6 kB)
  Using cached ml_dtypes-0.5.1-cp311-cp311-win_amd64.w

In [6]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# Open webcam
cap = cv2.VideoCapture(0)

def get_pose(landmarks):
    nose = landmarks.landmark[mp_pose.PoseLandmark.NOSE]
    left_shoulder = landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
    right_shoulder = landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
    left_wrist = landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST]
    right_wrist = landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST]
    left_elbow = landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW]
    right_elbow = landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW]
    
    pose_status = []

    # Head Up / Down detection (Using both shoulders for better accuracy)
    head_level = (left_shoulder.y + right_shoulder.y) / 2
    if nose.y < head_level - 0.05:
        pose_status.append("Head Up")
    elif nose.y > head_level + 0.02:  # Adjusted threshold for better detection
        pose_status.append("Head Down")
    
    # Hand Up detection (Using elbow as reference to avoid misclassification)
    if left_wrist.y < left_elbow.y and left_wrist.y < left_shoulder.y - 0.05:
        pose_status.append("Right Hand Up")
    if right_wrist.y < right_elbow.y and right_wrist.y < right_shoulder.y - 0.05:
        pose_status.append("Left Hand Up")
    
    return ", ".join(pose_status) if pose_status else "Neutral"

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)
    
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        pose_label = get_pose(results.pose_landmarks)
        cv2.putText(frame, pose_label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    cv2.imshow("Human Motion Detaction", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()